In [12]:
from helpers import database
import pandas as pd
import plotly.express as px

In [13]:
reviews = database.get_reviews(table="ff7remake_reviews_15_01_2025") # year 2026 - check fetch review
release_date = pd.Timestamp("2025-01-23") # ff7 rebirth published date
reviews["created"] = pd.to_datetime(reviews["timestamp_created"], unit="s")
reviews = reviews[
    (reviews["created"] >= release_date - pd.Timedelta(days=30)) &
    (reviews["created"] <= release_date + pd.Timedelta(days=30))
]


In [14]:
reviews.head()

,id,recommendation_id,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,language,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck,created
8590,188451547,None,76561199437816650,0,1,2817,0,250,1763671902,english,...,True,0,0,0.5,0,True,False,False,False,2025-02-21 23:50:15
8591,188449990,None,76561199145356041,342,65,1956,0,495,1766963906,english,...,True,0,0,0.5,0,False,False,False,False,2025-02-21 23:24:18
8592,188446292,None,76561198414060848,0,2,3076,0,800,1740538603,english,...,True,0,0,0.5,0,True,False,False,False,2025-02-21 22:22:37
8593,188440167,None,76561198029894651,0,34,3250,0,3250,1740171336,english,...,False,2,1,0.5,0,True,False,False,False,2025-02-21 20:56:20
8594,188438229,None,76561198044688336,334,18,2029,0,1406,1751515182,english,...,True,0,0,0.5,0,False,False,False,False,2025-02-21 20:28:40


In [15]:
daily_counts = reviews.groupby(reviews["created"].dt.date).size().reset_index(name="count")
daily_counts.head()

,created,count
0,2024-12-24,65
1,2024-12-25,57
2,2024-12-26,62
3,2024-12-27,65
4,2024-12-28,68


In [17]:
fig = px.bar(
    daily_counts,
    x="created",
    y="count",
    title="Daily review volume (±30 days from FF7 Rebirth release)",
    labels={"created": "Date", "count": "Reviews"}
)

fig.show()

In [6]:
daily_sentiment = reviews.groupby(reviews["created"].dt.date)["voted_up"].mean().reset_index(name="positivity")
daily_sentiment.head()

,created,positivity
0,2024-12-24,0.815385
1,2024-12-25,0.912281
2,2024-12-26,0.903226
3,2024-12-27,0.953846
4,2024-12-28,0.941176


In [8]:
fig = px.bar(
    daily_sentiment,
    x="created",
    y="positivity",
    title="Daily sentiment (±30 days from FF7 Rebirth release)",
    labels={"created": "Date", "positivity": "Sentiment"},
    color="positivity",
    color_continuous_scale=[(0, "red"), (1, "green")],
    range_color=[0, 1],
    range_y=[0, 1]
)
fig.update_yaxes(tickformat=".0%")

fig.show()

In [9]:
mention_mask = reviews["review"].str.contains(r"\brebirth\b", case=False, na=False)
mention_count = mention_mask.sum()
total_count = len(reviews)
positivity = reviews.loc[mention_mask, "voted_up"].mean()

print(f"Mentions of 'Rebirth': {mention_count} ({mention_count / total_count:.2%} of total)")
print(f"Positivity of those mentions: {positivity:.2%}")

mentions_daily = (
    reviews.loc[mention_mask]
    .groupby(reviews["created"].dt.date)
    .size()
    .reset_index(name="mentions")
)

Mentions of 'Rebirth': 338 (8.19% of total)
Positivity of those mentions: 88.17%


In [11]:
fig = px.bar(
    mentions_daily,
    x="created",
    y="mentions",
    title="Daily mentions of 'Rebirth' in FF7 Remake reviews",
    labels={"created": "Date", "mentions": "Mentions"}
)

fig.show()